# AI for FinOps: Complete Documentation and Implementation Guide

This notebook provides a comprehensive guide to implementing AI-powered FinOps solutions using AWS services, including Bedrock Agents, Lambda functions, MCP (Model Context Protocol), and an integrated AI chatbot.

## Table of Contents
1. [System Architecture Overview](#architecture)
2. [What is MCP (Model Context Protocol)?](#mcp)
3. [AWS Bedrock Agents Overview](#bedrock)
4. [Step-by-Step AWS CLI Deployment](#deployment)
5. [Lambda Functions Implementation](#lambda)
6. [AI Agent Code and Action Groups](#agents)
7. [How AWS Agents Work Together](#collaboration)
8. [Prompts and LLMs Configuration](#prompts)
9. [Complete Implementation Examples](#examples)
10. [Streamlit Dashboard with Integrated Chatbot](#chatbot)

## 1. System Architecture Overview <a name="architecture"></a>

The AI FinOps system consists of multiple components working together to provide intelligent cost optimization:

```
┌─────────────────────────────────────────────────────────────┐
│                     User Interface Layer                     │
│  (Web App / CLI / API Gateway)                             │
└─────────────────────┬───────────────────────────────────────┘
                      │
┌─────────────────────▼───────────────────────────────────────┐
│                  AWS Bedrock Agents                          │
│  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐        │
│  │Cost Analyzer│  │ Optimizer   │  │ Forecaster  │        │
│  │   Agent     │  │   Agent     │  │   Agent     │        │
│  └──────┬──────┘  └──────┬──────┘  └──────┬──────┘        │
└─────────┼────────────────┼────────────────┼────────────────┘
          │                │                │
┌─────────▼────────────────▼────────────────▼────────────────┐
│              Action Groups (Lambda Functions)               │
│  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐        │
│  │getCostData()│  │optimizeRes()│  │forecastCost│        │
│  │analyzeCost()│  │recommend()  │  │alertSetup()│        │
│  └─────────────┘  └─────────────┘  └─────────────┘        │
└─────────────────────┬───────────────────────────────────────┘
                      │
┌─────────────────────▼───────────────────────────────────────┐
│                    AWS Services Layer                        │
│  ┌──────────┐  ┌──────────┐  ┌──────────┐  ┌──────────┐   │
│  │  Cost    │  │    S3    │  │CloudWatch│  │   RDS    │   │
│  │ Explorer │  │          │  │          │  │          │   │
│  └──────────┘  └──────────┘  └──────────┘  └──────────┘   │
└─────────────────────────────────────────────────────────────┘
```

### Key Components:
- **Bedrock Agents**: AI-powered agents that understand natural language and execute actions
- **Lambda Functions**: Serverless compute that implements business logic
- **Action Groups**: Collections of Lambda functions grouped by functionality
- **AWS Services**: Native AWS services for data storage, monitoring, and cost analysis

## 2. What is MCP (Model Context Protocol)? <a name="mcp"></a>

MCP (Model Context Protocol) is a standardized protocol for sharing context between AI models and applications. In the context of AWS Bedrock:

### Key Features of MCP:
1. **Context Sharing**: Allows models to maintain conversation history and state
2. **Tool Integration**: Enables models to call external tools and APIs
3. **Security**: Provides secure context boundaries and access controls
4. **Scalability**: Supports distributed agent architectures

### MCP in AWS Bedrock:
```python
# Example MCP context structure for Bedrock
mcp_context = {
    "session_id": "unique-session-id",
    "user_context": {
        "account_id": "123456789",
        "permissions": ["read:costs", "write:recommendations"]
    },
    "conversation_history": [
        {"role": "user", "content": "What are my highest costs?"},
        {"role": "assistant", "content": "Your highest costs are in EC2..."}
    ],
    "available_tools": [
        "get_cost_data",
        "analyze_usage",
        "generate_recommendations"
    ]
}
```

## 3. AWS Bedrock Agents Overview <a name="bedrock"></a>

AWS Bedrock Agents are AI-powered assistants that can:
- Understand natural language requests
- Execute actions through Lambda functions
- Maintain conversation context
- Work with multiple foundation models

### Supported Foundation Models:
- **Anthropic Claude 3**: Advanced reasoning and analysis
- **Amazon Titan**: Cost-effective general purpose
- **AI21 Jurassic**: Specialized financial analysis

### Agent Capabilities:
1. **Natural Language Understanding**: Parse user queries
2. **Action Execution**: Call Lambda functions based on intent
3. **Context Management**: Remember previous interactions
4. **Multi-turn Conversations**: Handle complex workflows

## 4. Step-by-Step AWS CLI Deployment <a name="deployment"></a>

### Prerequisites Setup

In [ ]:
# Install AWS CLI if not already installed
!pip install awscli boto3

# Configure AWS credentials
import os
import boto3
import json

# Set your AWS region
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

# Get current AWS account ID
sts = boto3.client('sts')
account_id = sts.get_caller_identity()['Account']
print(f"AWS Account ID: {account_id}")

### Step 1: Create IAM Roles for Bedrock Agents and Lambda

In [ ]:
import json
import boto3
from botocore.exceptions import ClientError

iam = boto3.client('iam')

# Create Bedrock Agent Role
bedrock_trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

try:
    response = iam.create_role(
        RoleName='BedrockAgentRole',
        AssumeRolePolicyDocument=json.dumps(bedrock_trust_policy),
        Description='Role for Bedrock Agents'
    )
    print(f"Created Bedrock Agent Role: {response['Role']['Arn']}")
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Bedrock Agent Role already exists")
    else:
        raise

# Attach necessary policies
try:
    iam.attach_role_policy(
        RoleName='BedrockAgentRole',
        PolicyArn='arn:aws:iam::aws:policy/service-role/AmazonBedrockAgentBedrockFoundationModelPolicy'
    )
    print("Attached Bedrock Foundation Model Policy")
except Exception as e:
    print(f"Policy may already be attached: {e}")

# Create Lambda execution role
lambda_trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "lambda.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

try:
    response = iam.create_role(
        RoleName='FinOpsLambdaRole',
        AssumeRolePolicyDocument=json.dumps(lambda_trust_policy),
        Description='Role for FinOps Lambda functions'
    )
    print(f"Created Lambda Role: {response['Role']['Arn']}")
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Lambda Role already exists")
    else:
        raise

# Attach policies to Lambda role
policies_to_attach = [
    'arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole',
    'arn:aws:iam::aws:policy/AWSCostExplorerReadOnlyAccess',
    'arn:aws:iam::aws:policy/CloudWatchReadOnlyAccess'
]

for policy_arn in policies_to_attach:
    try:
        iam.attach_role_policy(
            RoleName='FinOpsLambdaRole',
            PolicyArn=policy_arn
        )
        print(f"Attached policy: {policy_arn.split('/')[-1]}")
    except Exception as e:
        print(f"Policy may already be attached: {e}")

### Step 2: Create S3 Bucket for Agent Instructions and Lambda Code

In [ ]:
import boto3
import time
from botocore.exceptions import ClientError

s3 = boto3.client('s3')

# Create unique bucket name
bucket_name = f"finops-bedrock-agents-{account_id}-{int(time.time())}"

try:
    s3.create_bucket(Bucket=bucket_name)
    print(f"Created S3 bucket: {bucket_name}")
    
    # Enable versioning
    s3.put_bucket_versioning(
        Bucket=bucket_name,
        VersioningConfiguration={'Status': 'Enabled'}
    )
    print("Enabled bucket versioning")
    
    # Store bucket name for later use
    os.environ['FINOPS_BUCKET'] = bucket_name
    
except ClientError as e:
    if e.response['Error']['Code'] == 'BucketAlreadyExists':
        print(f"Bucket {bucket_name} already exists")
    else:
        raise

## 5. Lambda Functions Implementation <a name="lambda"></a>

### Create Lambda Function Code

In [ ]:
import os
import zipfile

# Create directory for Lambda functions
lambda_dir = 'lambda_functions'
os.makedirs(lambda_dir, exist_ok=True)

# Cost Analysis Lambda Function
cost_analysis_code = '''import json
import boto3
from datetime import datetime, timedelta
from decimal import Decimal

ce_client = boto3.client('ce')  # Cost Explorer client
cloudwatch = boto3.client('cloudwatch')

def lambda_handler(event, context):
    """
    Handles cost analysis requests from Bedrock agent
    """
    print(f"Received event: {json.dumps(event)}")
    
    # Parse the action from Bedrock agent
    action = event.get('actionGroup', '')
    function = event.get('function', '')
    parameters = event.get('parameters', [])
    
    # Convert parameters list to dict
    params = {}
    for param in parameters:
        params[param.get('name', '')] = param.get('value', '')
    
    try:
        if function == 'get_cost_breakdown':
            result = get_cost_breakdown(params)
        elif function == 'analyze_cost_trends':
            result = analyze_cost_trends(params)
        elif function == 'identify_cost_anomalies':
            result = identify_cost_anomalies(params)
        else:
            result = {
                'error': f'Unknown function: {function}'
            }
        
        return {
            'response': result
        }
        
    except Exception as e:
        print(f"Error: {str(e)}")
        return {
            'response': {
                'error': str(e)
            }
        }

def get_cost_breakdown(params):
    """
    Get cost breakdown by service
    """
    days = int(params.get('days', '7'))
    end_date = datetime.now().date()
    start_date = end_date - timedelta(days=days)
    
    try:
        response = ce_client.get_cost_and_usage(
            TimePeriod={
                'Start': start_date.strftime('%Y-%m-%d'),
                'End': end_date.strftime('%Y-%m-%d')
            },
            Granularity='DAILY',
            Metrics=['UnblendedCost'],
            GroupBy=[
                {
                    'Type': 'DIMENSION',
                    'Key': 'SERVICE'
                }
            ]
        )
        
        # Process results
        cost_by_service = {}
        for result in response['ResultsByTime']:
            for group in result['Groups']:
                service = group['Keys'][0]
                cost = float(group['Metrics']['UnblendedCost']['Amount'])
                if service not in cost_by_service:
                    cost_by_service[service] = 0
                cost_by_service[service] += cost
        
        # Sort by cost
        sorted_costs = sorted(
            cost_by_service.items(), 
            key=lambda x: x[1], 
            reverse=True
        )
        
        return {
            'period': f'{days} days',
            'total_cost': sum(cost_by_service.values()),
            'cost_by_service': dict(sorted_costs[:10])  # Top 10 services
        }
        
    except Exception as e:
        return {
            'error': f'Failed to get cost breakdown: {str(e)}'
        }

def analyze_cost_trends(params):
    """
    Analyze cost trends and predict future costs
    """
    days = int(params.get('days', '30'))
    service = params.get('service', None)
    
    end_date = datetime.now().date()
    start_date = end_date - timedelta(days=days)
    
    try:
        # Build query parameters
        query_params = {
            'TimePeriod': {
                'Start': start_date.strftime('%Y-%m-%d'),
                'End': end_date.strftime('%Y-%m-%d')
            },
            'Granularity': 'DAILY',
            'Metrics': ['UnblendedCost']
        }
        
        if service:
            query_params['Filter'] = {
                'Dimensions': {
                    'Key': 'SERVICE',
                    'Values': [service]
                }
            }
        
        response = ce_client.get_cost_and_usage(**query_params)
        
        # Extract daily costs
        daily_costs = []
        for result in response['ResultsByTime']:
            date = result['TimePeriod']['Start']
            cost = float(result['Total']['UnblendedCost']['Amount'])
            daily_costs.append({
                'date': date,
                'cost': cost
            })
        
        # Calculate trend
        if len(daily_costs) > 1:
            first_week_avg = sum(d['cost'] for d in daily_costs[:7]) / 7
            last_week_avg = sum(d['cost'] for d in daily_costs[-7:]) / 7
            trend_percentage = ((last_week_avg - first_week_avg) / first_week_avg) * 100
        else:
            trend_percentage = 0
        
        return {
            'period': f'{days} days',
            'service': service or 'All Services',
            'daily_costs': daily_costs,
            'trend': {
                'percentage': round(trend_percentage, 2),
                'direction': 'increasing' if trend_percentage > 0 else 'decreasing'
            },
            'average_daily_cost': sum(d['cost'] for d in daily_costs) / len(daily_costs)
        }
        
    except Exception as e:
        return {
            'error': f'Failed to analyze trends: {str(e)}'
        }

def identify_cost_anomalies(params):
    """
    Identify unusual spikes or drops in costs
    """
    threshold = float(params.get('threshold', '20'))  # 20% change threshold
    
    end_date = datetime.now().date()
    start_date = end_date - timedelta(days=30)
    
    try:
        response = ce_client.get_cost_and_usage(
            TimePeriod={
                'Start': start_date.strftime('%Y-%m-%d'),
                'End': end_date.strftime('%Y-%m-%d')
            },
            Granularity='DAILY',
            Metrics=['UnblendedCost']
        )
        
        # Analyze daily costs for anomalies
        daily_costs = []
        for result in response['ResultsByTime']:
            cost = float(result['Total']['UnblendedCost']['Amount'])
            daily_costs.append(cost)
        
        # Calculate average and standard deviation
        avg_cost = sum(daily_costs) / len(daily_costs)
        
        anomalies = []
        for i, cost in enumerate(daily_costs):
            deviation = ((cost - avg_cost) / avg_cost) * 100
            if abs(deviation) > threshold:
                anomalies.append({
                    'date': (start_date + timedelta(days=i)).strftime('%Y-%m-%d'),
                    'cost': cost,
                    'deviation_percentage': round(deviation, 2),
                    'type': 'spike' if deviation > 0 else 'drop'
                })
        
        return {
            'anomalies_found': len(anomalies),
            'threshold_used': threshold,
            'average_daily_cost': avg_cost,
            'anomalies': anomalies
        }
        
    except Exception as e:
        return {
            'error': f'Failed to identify anomalies: {str(e)}'
        }
'''

# Write Lambda function to file
with open(os.path.join(lambda_dir, 'cost_analysis_lambda.py'), 'w') as f:
    f.write(cost_analysis_code)

print("Created cost_analysis_lambda.py")

In [ ]:
# Resource Optimization Lambda Function
optimization_code = '''import json
import boto3
from datetime import datetime, timedelta

ec2_client = boto3.client('ec2')
cloudwatch = boto3.client('cloudwatch')
rds_client = boto3.client('rds')

def lambda_handler(event, context):
    """
    Handles resource optimization requests from Bedrock agent
    """
    print(f"Received event: {json.dumps(event)}")
    
    function = event.get('function', '')
    parameters = event.get('parameters', [])
    
    # Convert parameters list to dict
    params = {}
    for param in parameters:
        params[param.get('name', '')] = param.get('value', '')
    
    try:
        if function == 'get_optimization_recommendations':
            result = get_optimization_recommendations(params)
        elif function == 'analyze_resource_utilization':
            result = analyze_resource_utilization(params)
        elif function == 'identify_idle_resources':
            result = identify_idle_resources(params)
        else:
            result = {'error': f'Unknown function: {function}'}
        
        return {'response': result}
        
    except Exception as e:
        print(f"Error: {str(e)}")
        return {'response': {'error': str(e)}}

def get_optimization_recommendations(params):
    """
    Get comprehensive optimization recommendations
    """
    resource_type = params.get('resource_type', 'all')
    recommendations = []
    
    # Get EC2 recommendations
    if resource_type in ['all', 'ec2']:
        ec2_recs = get_ec2_recommendations()
        recommendations.extend(ec2_recs)
    
    # Get RDS recommendations
    if resource_type in ['all', 'rds']:
        rds_recs = get_rds_recommendations()
        recommendations.extend(rds_recs)
    
    # Calculate total savings
    total_savings = sum(rec.get('estimated_monthly_savings', 0) for rec in recommendations)
    
    return {
        'total_recommendations': len(recommendations),
        'total_estimated_monthly_savings': total_savings,
        'recommendations': recommendations
    }

def get_ec2_recommendations():
    """
    Get EC2 optimization recommendations
    """
    recommendations = []
    
    try:
        # Get all running instances
        response = ec2_client.describe_instances(
            Filters=[{'Name': 'instance-state-name', 'Values': ['running']}]
        )
        
        for reservation in response['Reservations']:
            for instance in reservation['Instances']:
                instance_id = instance['InstanceId']
                instance_type = instance['InstanceType']
                
                # Get CPU utilization
                cpu_stats = get_instance_cpu_utilization(instance_id)
                
                if cpu_stats['average'] < 10:
                    recommendations.append({
                        'resource_type': 'EC2',
                        'resource_id': instance_id,
                        'current_type': instance_type,
                        'recommendation': 'Terminate or stop instance',
                        'reason': f"Low CPU utilization: {cpu_stats['average']:.1f}%",
                        'estimated_monthly_savings': estimate_instance_cost(instance_type),
                        'priority': 'high'
                    })
                elif cpu_stats['average'] < 40:
                    recommendations.append({
                        'resource_type': 'EC2',
                        'resource_id': instance_id,
                        'current_type': instance_type,
                        'recommendation': 'Consider downsizing instance',
                        'reason': f"Moderate CPU utilization: {cpu_stats['average']:.1f}%",
                        'estimated_monthly_savings': estimate_instance_cost(instance_type) * 0.3,
                        'priority': 'medium'
                    })
                    
    except Exception as e:
        print(f"Error getting EC2 recommendations: {e}")
    
    return recommendations

def get_instance_cpu_utilization(instance_id, days=7):
    """
    Get CPU utilization statistics for an EC2 instance
    """
    end_time = datetime.now()
    start_time = end_time - timedelta(days=days)
    
    try:
        response = cloudwatch.get_metric_statistics(
            Namespace='AWS/EC2',
            MetricName='CPUUtilization',
            Dimensions=[{'Name': 'InstanceId', 'Value': instance_id}],
            StartTime=start_time,
            EndTime=end_time,
            Period=3600,
            Statistics=['Average', 'Maximum']
        )
        
        datapoints = response.get('Datapoints', [])
        if datapoints:
            avg_cpu = sum(d['Average'] for d in datapoints) / len(datapoints)
            max_cpu = max(d['Maximum'] for d in datapoints)
            return {'average': avg_cpu, 'maximum': max_cpu}
        else:
            return {'average': 0, 'maximum': 0}
            
    except Exception as e:
        print(f"Error getting CPU stats: {e}")
        return {'average': 0, 'maximum': 0}

def estimate_instance_cost(instance_type):
    """
    Estimate monthly cost for an instance type
    """
    # Simplified cost estimation (in production, use AWS Pricing API)
    cost_map = {
        't2.micro': 8.5, 't2.small': 17, 't2.medium': 34,
        't3.micro': 7.6, 't3.small': 15.2, 't3.medium': 30.4,
        'm5.large': 70, 'm5.xlarge': 140,
        'c5.large': 62, 'c5.xlarge': 124
    }
    return cost_map.get(instance_type, 50)

def get_rds_recommendations():
    """
    Get RDS optimization recommendations
    """
    recommendations = []
    
    try:
        response = rds_client.describe_db_instances()
        
        for db in response['DBInstances']:
            db_id = db['DBInstanceIdentifier']
            
            # Check if Multi-AZ is needed for non-production
            if db['MultiAZ']:
                # Check tags to determine if production
                tags = rds_client.list_tags_for_resource(
                    ResourceName=db['DBInstanceArn']
                )['TagList']
                
                is_prod = any(tag['Key'] == 'Environment' and 
                             tag['Value'].lower() == 'production' 
                             for tag in tags)
                
                if not is_prod:
                    recommendations.append({
                        'resource_type': 'RDS',
                        'resource_id': db_id,
                        'recommendation': 'Disable Multi-AZ',
                        'reason': 'Non-production database with Multi-AZ enabled',
                        'estimated_monthly_savings': estimate_rds_cost(db) / 2,
                        'priority': 'high'
                    })
                    
    except Exception as e:
        print(f"Error getting RDS recommendations: {e}")
    
    return recommendations

def estimate_rds_cost(db_instance):
    """
    Estimate RDS monthly cost
    """
    instance_class = db_instance['DBInstanceClass']
    cost_map = {
        'db.t3.micro': 15, 'db.t3.small': 30, 'db.t3.medium': 60,
        'db.m5.large': 140, 'db.m5.xlarge': 280
    }
    return cost_map.get(instance_class, 100)

def identify_idle_resources(params):
    """
    Identify idle resources across services
    """
    idle_resources = []
    
    # Check EC2 instances
    try:
        response = ec2_client.describe_instances(
            Filters=[{'Name': 'instance-state-name', 'Values': ['running']}]
        )
        
        for reservation in response['Reservations']:
            for instance in reservation['Instances']:
                instance_id = instance['InstanceId']
                cpu_stats = get_instance_cpu_utilization(instance_id)
                
                if cpu_stats['average'] < 5:
                    idle_resources.append({
                        'resource_type': 'EC2',
                        'resource_id': instance_id,
                        'status': 'idle',
                        'details': f"CPU usage: {cpu_stats['average']:.1f}%",
                        'recommended_action': 'Terminate or stop'
                    })
                    
    except Exception as e:
        print(f"Error checking idle EC2: {e}")
    
    return {
        'total_idle_resources': len(idle_resources),
        'resources': idle_resources
    }
'''

# Write Lambda function to file
with open(os.path.join(lambda_dir, 'optimization_lambda.py'), 'w') as f:
    f.write(optimization_code)

print("Created optimization_lambda.py")

In [ ]:
# Forecasting Lambda Function
forecasting_code = '''import json
import boto3
from datetime import datetime, timedelta
import statistics

ce_client = boto3.client('ce')

def lambda_handler(event, context):
    """
    Handles cost forecasting requests from Bedrock agent
    """
    print(f"Received event: {json.dumps(event)}")
    
    function = event.get('function', '')
    parameters = event.get('parameters', [])
    
    # Convert parameters list to dict
    params = {}
    for param in parameters:
        params[param.get('name', '')] = param.get('value', '')
    
    try:
        if function == 'forecast_costs':
            result = forecast_costs(params)
        elif function == 'analyze_growth_trends':
            result = analyze_growth_trends(params)
        elif function == 'set_budget_alerts':
            result = set_budget_alerts(params)
        else:
            result = {'error': f'Unknown function: {function}'}
        
        return {'response': result}
        
    except Exception as e:
        print(f"Error: {str(e)}")
        return {'response': {'error': str(e)}}

def forecast_costs(params):
    """
    Forecast future costs based on historical data
    """
    months_to_forecast = int(params.get('months', '3'))
    
    # Get historical data (6 months)
    end_date = datetime.now().date()
    start_date = end_date - timedelta(days=180)
    
    try:
        response = ce_client.get_cost_and_usage(
            TimePeriod={
                'Start': start_date.strftime('%Y-%m-%d'),
                'End': end_date.strftime('%Y-%m-%d')
            },
            Granularity='MONTHLY',
            Metrics=['UnblendedCost']
        )
        
        # Extract monthly costs
        monthly_costs = []
        for result in response['ResultsByTime']:
            cost = float(result['Total']['UnblendedCost']['Amount'])
            monthly_costs.append(cost)
        
        # Calculate growth rate
        if len(monthly_costs) >= 2:
            growth_rates = []
            for i in range(1, len(monthly_costs)):
                if monthly_costs[i-1] > 0:
                    rate = (monthly_costs[i] - monthly_costs[i-1]) / monthly_costs[i-1]
                    growth_rates.append(rate)
            
            avg_growth_rate = statistics.mean(growth_rates) if growth_rates else 0
        else:
            avg_growth_rate = 0
        
        # Generate forecasts
        current_cost = monthly_costs[-1] if monthly_costs else 0
        forecasts = []
        
        for month in range(1, months_to_forecast + 1):
            forecasted_cost = current_cost * ((1 + avg_growth_rate) ** month)
            
            # Calculate confidence based on historical variance
            if len(monthly_costs) > 2:
                cost_variance = statistics.stdev(monthly_costs) / statistics.mean(monthly_costs)
                confidence = max(0.5, 1 - (cost_variance * month * 0.1))
            else:
                confidence = 0.7
            
            forecasts.append({
                'month': month,
                'predicted_cost': round(forecasted_cost, 2),
                'confidence': round(confidence, 2),
                'date': (end_date + timedelta(days=30*month)).strftime('%Y-%m')
            })
        
        return {
            'forecast_period': f'{months_to_forecast} months',
            'current_monthly_cost': round(current_cost, 2),
            'average_growth_rate': f'{avg_growth_rate*100:.1f}%',
            'forecasts': forecasts,
            'historical_data_points': len(monthly_costs)
        }
        
    except Exception as e:
        return {'error': f'Failed to forecast costs: {str(e)}'}

def analyze_growth_trends(params):
    """
    Analyze resource growth trends
    """
    service = params.get('service', None)
    
    end_date = datetime.now().date()
    start_date = end_date - timedelta(days=90)
    
    try:
        query_params = {
            'TimePeriod': {
                'Start': start_date.strftime('%Y-%m-%d'),
                'End': end_date.strftime('%Y-%m-%d')
            },
            'Granularity': 'MONTHLY',
            'Metrics': ['UnblendedCost'],
            'GroupBy': [{'Type': 'DIMENSION', 'Key': 'SERVICE'}]
        }
        
        response = ce_client.get_cost_and_usage(**query_params)
        
        # Analyze growth by service
        service_trends = {}
        
        for result in response['ResultsByTime']:
            for group in result['Groups']:
                svc = group['Keys'][0]
                cost = float(group['Metrics']['UnblendedCost']['Amount'])
                
                if svc not in service_trends:
                    service_trends[svc] = []
                service_trends[svc].append(cost)
        
        # Calculate growth rates
        growth_analysis = []
        for svc, costs in service_trends.items():
            if len(costs) >= 2 and costs[0] > 0:
                growth_rate = ((costs[-1] - costs[0]) / costs[0]) * 100
                growth_analysis.append({
                    'service': svc,
                    'initial_cost': round(costs[0], 2),
                    'current_cost': round(costs[-1], 2),
                    'growth_percentage': round(growth_rate, 1),
                    'trend': 'increasing' if growth_rate > 0 else 'decreasing'
                })
        
        # Sort by growth rate
        growth_analysis.sort(key=lambda x: x['growth_percentage'], reverse=True)
        
        return {
            'analysis_period': '3 months',
            'top_growing_services': growth_analysis[:5],
            'declining_services': [s for s in growth_analysis if s['growth_percentage'] < 0][:5]
        }
        
    except Exception as e:
        return {'error': f'Failed to analyze growth trends: {str(e)}'}

def set_budget_alerts(params):
    """
    Set up budget alerts (simulation)
    """
    budget_amount = float(params.get('amount', '10000'))
    alert_threshold = int(params.get('threshold', '80'))
    
    # In production, this would use AWS Budgets API
    # For demo, we'll return a simulated response
    
    return {
        'budget_created': True,
        'budget_details': {
            'name': 'FinOps-Monthly-Budget',
            'amount': budget_amount,
            'currency': 'USD',
            'time_period': 'MONTHLY',
            'alerts': [
                {
                    'threshold': alert_threshold,
                    'type': 'PERCENTAGE',
                    'notification': 'email'
                },
                {
                    'threshold': 100,
                    'type': 'PERCENTAGE',
                    'notification': 'email'
                }
            ]
        },
        'message': f'Budget alerts will trigger at {alert_threshold}% and 100% of ${budget_amount}'
    }
'''

# Write Lambda function to file
with open(os.path.join(lambda_dir, 'forecasting_lambda.py'), 'w') as f:
    f.write(forecasting_code)

print("Created forecasting_lambda.py")

### Package and Deploy Lambda Functions

In [ ]:
import os
import zipfile
import boto3
import time

lambda_client = boto3.client('lambda')
s3 = boto3.client('s3')

# Package Lambda functions
lambda_functions = [
    {
        'name': 'finops-cost-analysis',
        'file': 'cost_analysis_lambda.py',
        'handler': 'cost_analysis_lambda.lambda_handler'
    },
    {
        'name': 'finops-optimization',
        'file': 'optimization_lambda.py',
        'handler': 'optimization_lambda.lambda_handler'
    },
    {
        'name': 'finops-forecasting',
        'file': 'forecasting_lambda.py',
        'handler': 'forecasting_lambda.lambda_handler'
    }
]

deployed_functions = {}

for func in lambda_functions:
    # Create zip file
    zip_filename = f"{func['name']}.zip"
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(os.path.join(lambda_dir, func['file']), func['file'])
    
    # Upload to S3
    s3_key = f"lambda-functions/{zip_filename}"
    s3.upload_file(zip_filename, os.environ['FINOPS_BUCKET'], s3_key)
    print(f"Uploaded {zip_filename} to S3")
    
    # Deploy Lambda function
    try:
        response = lambda_client.create_function(
            FunctionName=func['name'],
            Runtime='python3.9',
            Role=f"arn:aws:iam::{account_id}:role/FinOpsLambdaRole",
            Handler=func['handler'],
            Code={
                'S3Bucket': os.environ['FINOPS_BUCKET'],
                'S3Key': s3_key
            },
            Timeout=60,
            MemorySize=256,
            Description=f'FinOps {func["name"]} function'
        )
        deployed_functions[func['name']] = response['FunctionArn']
        print(f"Deployed Lambda: {func['name']}")
        
    except lambda_client.exceptions.ResourceConflictException:
        # Function already exists, update it
        response = lambda_client.update_function_code(
            FunctionName=func['name'],
            S3Bucket=os.environ['FINOPS_BUCKET'],
            S3Key=s3_key
        )
        deployed_functions[func['name']] = response['FunctionArn']
        print(f"Updated Lambda: {func['name']}")
    
    # Clean up zip file
    os.remove(zip_filename)
    time.sleep(2)  # Small delay to avoid rate limits

print("\nDeployed Lambda functions:")
for name, arn in deployed_functions.items():
    print(f"  {name}: {arn}")

## 6. AI Agent Code and Action Groups <a name="agents"></a>

### Create Bedrock Agents with Action Groups

In [ ]:
# Create API schemas for action groups
import json

# Cost Analysis API Schema
cost_analysis_api_schema = {
    "openapi": "3.0.0",
    "info": {
        "title": "Cost Analysis API",
        "version": "1.0.0",
        "description": "API for analyzing AWS costs"
    },
    "paths": {
        "/get_cost_breakdown": {
            "post": {
                "summary": "Get cost breakdown by service",
                "description": "Returns breakdown of AWS costs by service for specified time period",
                "operationId": "getCostBreakdown",
                "requestBody": {
                    "content": {
                        "application/json": {
                            "schema": {
                                "type": "object",
                                "properties": {
                                    "days": {
                                        "type": "integer",
                                        "description": "Number of days to analyze",
                                        "default": 7
                                    }
                                }
                            }
                        }
                    }
                },
                "responses": {
                    "200": {
                        "description": "Cost breakdown retrieved successfully"
                    }
                }
            }
        },
        "/analyze_cost_trends": {
            "post": {
                "summary": "Analyze cost trends",
                "description": "Analyze cost trends over time and identify patterns",
                "operationId": "analyzeCostTrends",
                "requestBody": {
                    "content": {
                        "application/json": {
                            "schema": {
                                "type": "object",
                                "properties": {
                                    "days": {
                                        "type": "integer",
                                        "default": 30
                                    },
                                    "service": {
                                        "type": "string",
                                        "description": "Specific service to analyze"
                                    }
                                }
                            }
                        }
                    }
                },
                "responses": {
                    "200": {
                        "description": "Trend analysis completed"
                    }
                }
            }
        },
        "/identify_cost_anomalies": {
            "post": {
                "summary": "Identify cost anomalies",
                "description": "Find unusual spikes or drops in costs",
                "operationId": "identifyCostAnomalies",
                "requestBody": {
                    "content": {
                        "application/json": {
                            "schema": {
                                "type": "object",
                                "properties": {
                                    "threshold": {
                                        "type": "number",
                                        "default": 20
                                    }
                                }
                            }
                        }
                    }
                },
                "responses": {
                    "200": {
                        "description": "Anomalies identified"
                    }
                }
            }
        }
    }
}

# Save to S3
s3.put_object(
    Bucket=os.environ['FINOPS_BUCKET'],
    Key='api-schemas/cost-analysis-api.json',
    Body=json.dumps(cost_analysis_api_schema)
)

print("Created Cost Analysis API schema")

In [ ]:
# Optimization API Schema
optimization_api_schema = {
    "openapi": "3.0.0",
    "info": {
        "title": "Resource Optimization API",
        "version": "1.0.0"
    },
    "paths": {
        "/get_optimization_recommendations": {
            "post": {
                "summary": "Get resource optimization recommendations",
                "operationId": "getOptimizationRecommendations",
                "requestBody": {
                    "content": {
                        "application/json": {
                            "schema": {
                                "type": "object",
                                "properties": {
                                    "resource_type": {
                                        "type": "string",
                                        "enum": ["ec2", "rds", "s3", "all"],
                                        "default": "all"
                                    }
                                }
                            }
                        }
                    }
                },
                "responses": {
                    "200": {
                        "description": "Recommendations retrieved"
                    }
                }
            }
        },
        "/identify_idle_resources": {
            "post": {
                "summary": "Identify idle resources",
                "operationId": "identifyIdleResources",
                "responses": {
                    "200": {
                        "description": "Idle resources identified"
                    }
                }
            }
        }
    }
}

# Forecasting API Schema
forecasting_api_schema = {
    "openapi": "3.0.0",
    "info": {
        "title": "Cost Forecasting API",
        "version": "1.0.0"
    },
    "paths": {
        "/forecast_costs": {
            "post": {
                "summary": "Forecast future costs",
                "operationId": "forecastCosts",
                "requestBody": {
                    "content": {
                        "application/json": {
                            "schema": {
                                "type": "object",
                                "properties": {
                                    "months": {
                                        "type": "integer",
                                        "default": 3
                                    }
                                }
                            }
                        }
                    }
                },
                "responses": {
                    "200": {
                        "description": "Forecast generated"
                    }
                }
            }
        },
        "/analyze_growth_trends": {
            "post": {
                "summary": "Analyze growth trends",
                "operationId": "analyzeGrowthTrends",
                "requestBody": {
                    "content": {
                        "application/json": {
                            "schema": {
                                "type": "object",
                                "properties": {
                                    "service": {
                                        "type": "string"
                                    }
                                }
                            }
                        }
                    }
                },
                "responses": {
                    "200": {
                        "description": "Growth trends analyzed"
                    }
                }
            }
        }
    }
}

# Save schemas to S3
s3.put_object(
    Bucket=os.environ['FINOPS_BUCKET'],
    Key='api-schemas/optimization-api.json',
    Body=json.dumps(optimization_api_schema)
)

s3.put_object(
    Bucket=os.environ['FINOPS_BUCKET'],
    Key='api-schemas/forecasting-api.json',
    Body=json.dumps(forecasting_api_schema)
)

print("Created Optimization and Forecasting API schemas")

In [ ]:
import boto3
import time
from botocore.exceptions import ClientError

bedrock_agent = boto3.client('bedrock-agent')

# Agent instructions
agent_instructions = {
    'cost_analyzer': '''You are a FinOps Cost Analysis specialist AI assistant. Your primary role is to help users understand their AWS costs.

Your capabilities include:
1. Analyzing AWS costs by service, region, and time period
2. Identifying cost trends and patterns
3. Detecting cost anomalies and unusual spending
4. Providing detailed breakdowns of cloud expenses

When users ask about costs:
- Always provide specific numbers and percentages
- Identify the top cost drivers
- Compare current costs to historical data when relevant
- Highlight any concerning trends or anomalies

Use the available functions to fetch real-time cost data and provide accurate insights.''',
    
    'optimizer': '''You are an AWS Resource Optimization specialist focused on helping users reduce cloud costs.

Your capabilities include:
1. Identifying underutilized or idle resources
2. Recommending right-sizing opportunities
3. Suggesting cost-saving architectural changes
4. Calculating potential savings from optimizations

When providing recommendations:
- Include specific resource IDs and configurations
- Provide estimated monthly and annual savings
- Assess performance impact and implementation difficulty
- Prioritize by potential savings and ease of implementation

Always validate recommendations against actual usage patterns.''',
    
    'forecaster': '''You are an AWS Cost Forecasting expert specializing in predictive analytics.

Your capabilities include:
1. Forecasting future costs based on historical trends
2. Identifying growth patterns and seasonality
3. Setting up budget alerts and thresholds
4. Predicting cost impacts of planned changes

When providing forecasts:
- Base predictions on historical data and trends
- Include confidence levels with predictions
- Account for seasonal variations and growth rates
- Recommend appropriate budget thresholds

Help users plan their cloud spending proactively.'''
}

# Create agents
created_agents = {}

agent_configs = [
    {
        'name': 'FinOps-CostAnalyzer',
        'type': 'cost_analyzer',
        'model': 'anthropic.claude-3-sonnet-20240229-v1:0'
    },
    {
        'name': 'FinOps-Optimizer',
        'type': 'optimizer',
        'model': 'anthropic.claude-3-sonnet-20240229-v1:0'
    },
    {
        'name': 'FinOps-Forecaster',
        'type': 'forecaster',
        'model': 'anthropic.claude-3-sonnet-20240229-v1:0'
    }
]

for config in agent_configs:
    try:
        # Create the agent
        response = bedrock_agent.create_agent(
            agentName=config['name'],
            agentResourceRoleArn=f"arn:aws:iam::{account_id}:role/BedrockAgentRole",
            description=f"AI-powered {config['type'].replace('_', ' ')} for FinOps",
            idleSessionTTLInSeconds=1800,
            foundationModel=config['model'],
            instruction=agent_instructions[config['type']],
            tags={
                'Project': 'FinOps',
                'Type': config['type']
            }
        )
        
        agent_id = response['agent']['agentId']
        created_agents[config['type']] = {
            'id': agent_id,
            'name': config['name'],
            'arn': response['agent']['agentArn']
        }
        
        print(f"Created agent: {config['name']} (ID: {agent_id})")
        
        # Prepare the agent
        bedrock_agent.prepare_agent(agentId=agent_id)
        print(f"Prepared agent: {config['name']}")
        
        time.sleep(5)  # Wait for agent preparation
        
    except ClientError as e:
        print(f"Error creating agent {config['name']}: {e}")
        if e.response['Error']['Code'] == 'ConflictException':
            print("Agent may already exist")

print("\nCreated agents:")
for agent_type, agent_info in created_agents.items():
    print(f"  {agent_type}: {agent_info['name']} (ID: {agent_info['id']})")

### Create Action Groups for Agents

In [ ]:
# Create action groups for each agent
action_group_configs = [
    {
        'agent_type': 'cost_analyzer',
        'action_group_name': 'CostAnalysisActions',
        'lambda_arn': deployed_functions['finops-cost-analysis'],
        'api_schema_key': 'api-schemas/cost-analysis-api.json'
    },
    {
        'agent_type': 'optimizer',
        'action_group_name': 'OptimizationActions',
        'lambda_arn': deployed_functions['finops-optimization'],
        'api_schema_key': 'api-schemas/optimization-api.json'
    },
    {
        'agent_type': 'forecaster',
        'action_group_name': 'ForecastingActions',
        'lambda_arn': deployed_functions['finops-forecasting'],
        'api_schema_key': 'api-schemas/forecasting-api.json'
    }
]

for config in action_group_configs:
    if config['agent_type'] in created_agents:
        agent_id = created_agents[config['agent_type']]['id']
        
        try:
            # Create action group
            response = bedrock_agent.create_agent_action_group(
                agentId=agent_id,
                agentVersion='DRAFT',
                actionGroupName=config['action_group_name'],
                actionGroupExecutor={
                    'lambda': config['lambda_arn']
                },
                apiSchema={
                    's3': {
                        's3BucketName': os.environ['FINOPS_BUCKET'],
                        's3ObjectKey': config['api_schema_key']
                    }
                },
                description=f"Action group for {config['agent_type'].replace('_', ' ')}"
            )
            
            print(f"Created action group '{config['action_group_name']}' for {config['agent_type']}")
            
            # Prepare agent again after adding action group
            bedrock_agent.prepare_agent(agentId=agent_id)
            print(f"Re-prepared agent after adding action group")
            
            time.sleep(5)
            
        except ClientError as e:
            print(f"Error creating action group: {e}")

### Create Agent Aliases for Production Use

In [ ]:
# Create aliases for agents
agent_aliases = {}

for agent_type, agent_info in created_agents.items():
    try:
        response = bedrock_agent.create_agent_alias(
            agentId=agent_info['id'],
            agentAliasName='production',
            description=f"Production alias for {agent_type}",
            tags={
                'Environment': 'production',
                'Type': agent_type
            }
        )
        
        agent_aliases[agent_type] = response['agentAlias']['agentAliasId']
        print(f"Created production alias for {agent_type}: {response['agentAlias']['agentAliasId']}")
        
    except ClientError as e:
        print(f"Error creating alias: {e}")

# Store agent information for later use
finops_agents = {
    agent_type: {
        'id': agent_info['id'],
        'name': agent_info['name'],
        'alias': agent_aliases.get(agent_type, 'TSTALIASID'),
        'arn': agent_info['arn']
    }
    for agent_type, agent_info in created_agents.items()
}

print("\nFinOps Agents Configuration:")
print(json.dumps(finops_agents, indent=2))

## 7. Test the Deployed Agents

In [ ]:
import boto3
import uuid

bedrock_runtime = boto3.client('bedrock-agent-runtime')

def invoke_agent(agent_type, prompt):
    """
    Invoke a Bedrock agent with a prompt
    """
    if agent_type not in finops_agents:
        print(f"Unknown agent type: {agent_type}")
        return None
    
    agent_config = finops_agents[agent_type]
    session_id = str(uuid.uuid4())
    
    try:
        response = bedrock_runtime.invoke_agent(
            agentId=agent_config['id'],
            agentAliasId=agent_config['alias'],
            sessionId=session_id,
            inputText=prompt
        )
        
        # Process response
        result = ""
        for event in response.get('completion', []):
            if 'chunk' in event:
                chunk = event['chunk']
                if 'bytes' in chunk:
                    result += chunk['bytes'].decode('utf-8')
        
        return result
        
    except Exception as e:
        print(f"Error invoking agent: {e}")
        return None

# Test each agent
test_prompts = {
    'cost_analyzer': "What are my top 5 AWS services by cost in the last 7 days?",
    'optimizer': "Find idle EC2 instances and recommend optimization actions",
    'forecaster': "Forecast my AWS costs for the next 3 months based on current trends"
}

print("Testing deployed agents...\n")

for agent_type, prompt in test_prompts.items():
    if agent_type in finops_agents:
        print(f"Testing {agent_type} agent...")
        print(f"Prompt: {prompt}")
        
        result = invoke_agent(agent_type, prompt)
        if result:
            print(f"Response: {result[:500]}...\n")  # Show first 500 chars
        else:
            print("No response received\n")

## 8. Multi-Agent Orchestration Example

In [ ]:
class FinOpsMultiAgentOrchestrator:
    """
    Orchestrates multiple Bedrock agents for complex FinOps tasks
    """
    
    def __init__(self, agents_config):
        self.bedrock_runtime = boto3.client('bedrock-agent-runtime')
        self.agents = agents_config
        self.session_attributes = {}
    
    def process_complex_request(self, user_request):
        """
        Process a complex request that requires multiple agents
        """
        print(f"Processing complex request: {user_request}\n")
        
        # Step 1: Analyze current costs
        print("Step 1: Analyzing current costs...")
        cost_analysis = self._invoke_agent(
            'cost_analyzer',
            "Analyze my AWS costs for the last 30 days and identify the top cost drivers"
        )
        print(f"Cost Analysis: {cost_analysis[:300]}...\n")
        
        # Step 2: Get optimization recommendations based on cost analysis
        print("Step 2: Getting optimization recommendations...")
        optimization_prompt = f"""Based on the cost analysis showing high costs, 
        provide specific optimization recommendations for reducing costs.
        Focus on the highest cost services."""
        
        optimizations = self._invoke_agent('optimizer', optimization_prompt)
        print(f"Optimizations: {optimizations[:300]}...\n")
        
        # Step 3: Forecast savings impact
        print("Step 3: Forecasting cost impact...")
        forecast_prompt = """Based on the optimization recommendations, 
        forecast the potential cost savings over the next 3 months 
        if these optimizations are implemented."""
        
        forecast = self._invoke_agent('forecaster', forecast_prompt)
        print(f"Forecast: {forecast[:300]}...\n")
        
        # Synthesize results
        return {
            'cost_analysis': cost_analysis,
            'optimizations': optimizations,
            'forecast': forecast,
            'summary': self._create_summary(cost_analysis, optimizations, forecast)
        }
    
    def _invoke_agent(self, agent_type, prompt):
        """
        Invoke a specific agent
        """
        agent_config = self.agents.get(agent_type)
        if not agent_config:
            return "Agent not found"
        
        session_id = self.session_attributes.get('session_id', str(uuid.uuid4()))
        self.session_attributes['session_id'] = session_id
        
        try:
            response = self.bedrock_runtime.invoke_agent(
                agentId=agent_config['id'],
                agentAliasId=agent_config['alias'],
                sessionId=session_id,
                inputText=prompt
            )
            
            result = ""
            for event in response.get('completion', []):
                if 'chunk' in event:
                    chunk = event['chunk']
                    if 'bytes' in chunk:
                        result += chunk['bytes'].decode('utf-8')
            
            return result
            
        except Exception as e:
            return f"Error: {str(e)}"
    
    def _create_summary(self, cost_analysis, optimizations, forecast):
        """
        Create a summary of the multi-agent analysis
        """
        return f"""FinOps Analysis Summary:

1. Current State: The cost analysis reveals your AWS spending patterns and identifies key cost drivers.

2. Optimization Opportunities: Multiple optimization strategies have been identified that could significantly reduce your AWS costs.

3. Projected Impact: If implemented, these optimizations are forecasted to provide substantial cost savings over the coming months.

This multi-agent analysis provides a comprehensive view of your current costs, actionable optimization strategies, and projected savings."""

# Test multi-agent orchestration
orchestrator = FinOpsMultiAgentOrchestrator(finops_agents)

complex_request = """I need a comprehensive FinOps analysis: 
1. What are my current AWS costs and trends?
2. Where can I optimize to reduce costs?
3. What would be the impact of these optimizations?"""

print("\n" + "="*60)
print("Multi-Agent FinOps Analysis")
print("="*60 + "\n")

results = orchestrator.process_complex_request(complex_request)

print("\n" + "="*60)
print("Analysis Summary")
print("="*60)
print(results['summary'])

## 9. Summary and Next Steps

### What We've Accomplished

1. **Created IAM Roles**: Set up necessary permissions for Bedrock agents and Lambda functions
2. **Deployed Lambda Functions**: Created and deployed three Lambda functions for cost analysis, optimization, and forecasting
3. **Created Bedrock Agents**: Deployed three specialized AI agents with specific prompts and models
4. **Configured Action Groups**: Connected agents to Lambda functions through API schemas
5. **Tested Integration**: Verified that agents can invoke Lambda functions and return results
6. **Demonstrated Multi-Agent Orchestration**: Showed how multiple agents can work together

### Architecture Summary

```
User Request
     ↓
Bedrock Agent (with LLM)
     ↓
Action Group (API Schema)
     ↓
Lambda Function
     ↓
AWS Services (Cost Explorer, EC2, CloudWatch, etc.)
```

### Key Components Created

1. **Lambda Functions**:
   - `finops-cost-analysis`: Analyzes costs and identifies anomalies
   - `finops-optimization`: Finds optimization opportunities
   - `finops-forecasting`: Predicts future costs

2. **Bedrock Agents**:
   - **Cost Analyzer**: Specialized in cost analysis and reporting
   - **Optimizer**: Focused on resource optimization recommendations
   - **Forecaster**: Handles cost predictions and budget planning

3. **Action Groups**: Each agent has its own action group connecting to the appropriate Lambda function

### Next Steps

1. **Enhance Lambda Functions**: Add more sophisticated analysis and optimization logic
2. **Integrate with Your Tools**: Connect to your existing FinOps dashboards and tools
3. **Add More Agents**: Create specialized agents for specific use cases (e.g., security cost analysis)
4. **Implement Feedback Loop**: Use agent outputs to continuously improve recommendations
5. **Set Up Monitoring**: Track agent usage, costs, and performance
6. **Create UI**: Build a web interface or Slack bot for easier interaction

### Save Configuration for Future Use

In [ ]:
# Save configuration for future use
config = {
    'bucket': os.environ['FINOPS_BUCKET'],
    'region': os.environ['AWS_DEFAULT_REGION'],
    'agents': finops_agents,
    'lambda_functions': deployed_functions,
    'account_id': account_id
}

# Save to file
with open('finops_config.json', 'w') as f:
    json.dump(config, f, indent=2)

print("Configuration saved to finops_config.json")
print("\nYour FinOps AI Platform is now fully deployed and ready to use!")

## 10. Streamlit Dashboard with Integrated Chatbot <a name="chatbot"></a>

### Overview
The enhanced FinOps dashboard now includes an integrated AI-powered chatbot that can:
- Answer questions about AWS costs in natural language
- Provide real-time cost analysis using cached data
- Integrate with Bedrock agents for advanced insights
- Work seamlessly with Lambda functions and MCP

### Key Features of the Chatbot Integration

1. **Natural Language Interface**: Users can ask questions like:
   - "What are my highest costs this month?"
   - "Which services are growing fastest?"
   - "How can I reduce my EC2 costs?"

2. **Real-time Data Integration**: The chatbot has access to:
   - Current cost data from AWS Cost Explorer
   - EC2 utilization metrics
   - Historical cost trends
   - Optimization recommendations

3. **AI-Powered Responses**: Uses either:
   - AWS Bedrock agents for advanced analysis
   - Fallback responses with real data when AI services are unavailable

4. **Context Awareness**: Maintains conversation history and understands follow-up questions

### Chatbot Implementation Code

The chatbot is implemented in the enhanced dashboard (`finops_dashboard_with_chatbot.py`). Here's how it works:

In [ ]:
# Key chatbot components from finops_dashboard_with_chatbot.py

# 1. Session State Initialization
def initialize_chat_session():
    """Initialize chat session state"""
    if 'chat_messages' not in st.session_state:
        st.session_state.chat_messages = []
    if 'cost_data_cache' not in st.session_state:
        st.session_state.cost_data_cache = None
    if 'chat_mode' not in st.session_state:
        st.session_state.chat_mode = False

# 2. Bedrock Agent Integration
def query_bedrock_agent(prompt):
    """Query Bedrock agent for AI insights"""
    if not AGENT_ID or not AGENT_ALIAS:
        return "Bedrock agent not configured. Using fallback analysis."
    
    try:
        session_id = str(uuid.uuid4())
        
        response = bedrock_runtime.invoke_agent(
            agentId=AGENT_ID,
            agentAliasId=AGENT_ALIAS,
            sessionId=session_id,
            inputText=prompt
        )
        
        # Process streaming response
        result = ""
        for event in response.get('completion', []):
            if 'chunk' in event:
                chunk = event['chunk']
                if 'bytes' in chunk:
                    result += chunk['bytes'].decode('utf-8')
        
        return result if result else "No response from agent."
        
    except Exception as e:
        return f"Using fallback analysis due to: {str(e)}"

# 3. Lambda Function Integration
def invoke_lambda_for_insights(function_name, parameters):
    """Invoke Lambda function for specific insights"""
    event = {
        'apiPath': f'/{function_name}',
        'parameters': parameters,
        'actionGroup': 'chat',
        'httpMethod': 'POST'
    }
    
    try:
        response = lambda_client.invoke(
            FunctionName='finops-cost-analysis',
            InvocationType='RequestResponse',
            Payload=json.dumps(event)
        )
        
        result = json.loads(response['Payload'].read())
        
        if 'response' in result and 'responseBody' in result['response']:
            body = result['response']['responseBody']['application/json']['body']
            return json.loads(body)
        else:
            return result
            
    except Exception as e:
        return {'error': str(e)}

# 4. Fallback Response Generator
def generate_fallback_response(prompt, cost_data):
    """Generate response using cached data when AI is unavailable"""
    prompt_lower = prompt.lower()
    
    if not cost_data:
        return "I don't have any cost data available. Please refresh the dashboard."
    
    # Analyze prompt intent
    if 'highest' in prompt_lower or 'top' in prompt_lower:
        # Get top services by cost
        services = cost_data.get('services_by_cost', {})
        top_5 = list(services.items())[:5]
        
        response = "Based on your current data, here are your top 5 services by cost:\\n\\n"
        for i, (service, cost) in enumerate(top_5, 1):
            response += f"{i}. **{service}**: ${cost:,.2f}\\n"
        
        response += f"\\nTotal cost for the period: ${cost_data.get('total_cost', 0):,.2f}"
        
    elif 'save' in prompt_lower or 'reduce' in prompt_lower:
        # Provide optimization tips
        response = "Here are some cost optimization recommendations:\\n\\n"
        response += "1. **Right-size EC2 instances** - Review underutilized instances\\n"
        response += "2. **Use Reserved Instances** - Save up to 72% on predictable workloads\\n"
        response += "3. **Enable S3 lifecycle policies** - Move old data to cheaper storage\\n"
        response += "4. **Review unattached EBS volumes** - Delete unused storage\\n"
        response += "5. **Use Spot instances** - Save up to 90% on fault-tolerant workloads"
        
    elif 'trend' in prompt_lower or 'growing' in prompt_lower:
        # Analyze trends
        daily_trend = cost_data.get('daily_trend', [])
        if len(daily_trend) > 1:
            first_cost = daily_trend[0]['cost']
            last_cost = daily_trend[-1]['cost']
            trend_pct = ((last_cost - first_cost) / first_cost * 100) if first_cost > 0 else 0
            
            response = f"Your costs are {'increasing' if trend_pct > 0 else 'decreasing'} "
            response += f"by {abs(trend_pct):.1f}% over the period.\\n\\n"
            response += f"First day: ${first_cost:,.2f}\\n"
            response += f"Last day: ${last_cost:,.2f}\\n"
            response += f"Daily average: ${cost_data.get('daily_average', 0):,.2f}"
        else:
            response = "Not enough data to analyze trends."
            
    else:
        # General cost summary
        response = f"Here's your cost summary:\\n\\n"
        response += f"**Total cost**: ${cost_data.get('total_cost', 0):,.2f}\\n"
        response += f"**Daily average**: ${cost_data.get('daily_average', 0):,.2f}\\n"
        response += f"**Number of services**: {cost_data.get('service_count', 0)}\\n\\n"
        response += "Ask me about specific services, trends, or optimization opportunities!"
    
    return response

### Dashboard Structure with Chatbot

The enhanced dashboard includes 6 main tabs:

In [ ]:
# Dashboard Tab Structure
tabs = st.tabs([
    "📊 Cost Overview",      # Real-time cost analysis
    "📈 Trends",            # Cost trends and patterns
    "🖥️ EC2 Analysis",      # Instance utilization
    "💡 Optimizations",     # Cost savings recommendations  
    "🤖 AI Chat",          # NEW: Dedicated chatbot interface
    "🧪 Test Lambda"       # Lambda function testing
])

# AI Chat Tab Implementation
with tabs[4]:  # AI Chat tab
    st.header("🤖 AI FinOps Assistant")
    st.markdown("Ask me anything about your AWS costs!")
    
    # Chat interface
    chat_container = st.container()
    
    with chat_container:
        # Display chat history
        for msg in st.session_state.chat_messages:
            with st.chat_message(msg["role"]):
                st.markdown(msg["content"])
    
    # Chat input
    if prompt := st.chat_input("Ask about your AWS costs..."):
        # Add user message
        st.session_state.chat_messages.append({"role": "user", "content": prompt})
        
        with st.chat_message("user"):
            st.markdown(prompt)
        
        # Get AI response
        with st.chat_message("assistant"):
            with st.spinner("Analyzing..."):
                # Update cost data cache if needed
                if not st.session_state.cost_data_cache:
                    cost_data = get_cost_data(7)
                    if cost_data:
                        # Process and cache the data
                        st.session_state.cost_data_cache = process_cost_data(cost_data)
                
                # Try Bedrock agent first
                ai_response = query_bedrock_agent(prompt)
                
                # If Bedrock fails, use fallback with real data
                if "fallback" in ai_response.lower() or "error" in ai_response.lower():
                    ai_response = generate_fallback_response(
                        prompt, 
                        st.session_state.cost_data_cache
                    )
                
                st.markdown(ai_response)
                
        # Add assistant message to history
        st.session_state.chat_messages.append({
            "role": "assistant", 
            "content": ai_response
        })

### Enhanced Chat Mode

The dashboard also includes an enhanced chat mode that can be toggled from the sidebar:

In [ ]:
# Enhanced Chat Mode Feature
with st.sidebar:
    st.header("Configuration")
    days = st.slider("Analysis Period (days)", 1, 90, 7)
    
    # NEW: Enhanced chat mode toggle
    st.markdown("---")
    st.markdown("### 💬 Chat Mode")
    chat_mode = st.toggle(
        "Enable Enhanced Chat",
        value=st.session_state.get('chat_mode', False),
        help="Toggle to focus on AI chat interface"
    )
    
    if chat_mode != st.session_state.get('chat_mode', False):
        st.session_state.chat_mode = chat_mode
        st.rerun()
    
    if chat_mode:
        st.info("Chat mode enabled! The AI assistant has full context of your cost data.")
        
        # Quick prompts in chat mode
        st.markdown("#### Quick Prompts")
        quick_prompts = [
            "What are my top 5 costs?",
            "How can I reduce EC2 costs?",
            "Show me cost trends",
            "Find idle resources",
            "Forecast next month's costs"
        ]
        
        for prompt in quick_prompts:
            if st.button(f"📝 {prompt}", key=f"quick_{prompt}"):
                # This would add the prompt to the chat
                st.session_state.pending_prompt = prompt
                st.rerun()

### Export Functionality

The chatbot-enhanced dashboard includes comprehensive export capabilities:

In [ ]:
# Export Functionality
def export_data(format_type, data):
    """Export dashboard data in various formats"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    if format_type == "CSV":
        # Prepare CSV data
        df = pd.DataFrame()
        
        # Add cost summary
        if 'total_cost' in data:
            df['Metric'] = ['Total Cost', 'Daily Average', 'Service Count']
            df['Value'] = [
                f"${data['total_cost']:,.2f}",
                f"${data['daily_average']:,.2f}",
                data['service_count']
            ]
        
        # Add services data
        if 'services_by_cost' in data:
            services_df = pd.DataFrame(
                list(data['services_by_cost'].items()),
                columns=['Service', 'Cost']
            )
            services_df['Cost'] = services_df['Cost'].apply(lambda x: f"${x:,.2f}")
        
        # Export CSV
        csv_buffer = io.StringIO()
        df.to_csv(csv_buffer, index=False)
        csv_data = csv_buffer.getvalue()
        
        st.download_button(
            label="📥 Download CSV",
            data=csv_data,
            file_name=f"finops_report_{timestamp}.csv",
            mime="text/csv"
        )
    
    elif format_type == "JSON":
        # Export full data as JSON
        json_data = json.dumps(data, indent=2, default=str)
        
        st.download_button(
            label="📥 Download JSON",
            data=json_data,
            file_name=f"finops_report_{timestamp}.json",
            mime="application/json"
        )
    
    elif format_type == "PDF":
        # Generate PDF summary
        pdf_content = f\"\"\"
# FinOps Cost Report
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Executive Summary
- Total Cost: ${data.get('total_cost', 0):,.2f}
- Daily Average: ${data.get('daily_average', 0):,.2f}
- Number of Services: {data.get('service_count', 0)}
- Analysis Period: {data.get('period', 'N/A')}

## Top Services by Cost
\"\"\"
        if 'services_by_cost' in data:
            for i, (service, cost) in enumerate(
                list(data['services_by_cost'].items())[:10], 1
            ):
                pdf_content += f"{i}. {service}: ${cost:,.2f}\\n"
        
        pdf_content += \"\"\"
## Cost Optimization Recommendations
1. Review and right-size underutilized EC2 instances
2. Implement Reserved Instances for stable workloads
3. Enable S3 lifecycle policies for data archival
4. Review and delete unattached EBS volumes
5. Consider using Spot instances for fault-tolerant applications

## AI Assistant Insights
The integrated AI chatbot can provide:
- Real-time cost analysis and trends
- Personalized optimization recommendations
- Cost forecasting and budget planning
- Resource utilization insights
\"\"\"
        
        # For demo, show PDF content
        st.text_area("PDF Preview", pdf_content, height=400)
        st.info("Full PDF export functionality would be implemented with reportlab or similar library")

# Export section in sidebar
with st.sidebar:
    st.markdown("---")
    st.markdown("### 📤 Export Data")
    
    export_format = st.selectbox(
        "Export Format",
        ["CSV", "JSON", "PDF Summary"]
    )
    
    if st.button("Export Report"):
        # Prepare export data
        export_data_dict = {
            'total_cost': st.session_state.get('total_cost', 0),
            'daily_average': st.session_state.get('daily_average', 0),
            'service_count': st.session_state.get('service_count', 0),
            'services_by_cost': st.session_state.get('services_by_cost', {}),
            'period': f"{days} days",
            'export_date': datetime.now().isoformat(),
            'chat_history': st.session_state.get('chat_messages', [])
        }
        
        if export_format == "PDF Summary":
            export_data("PDF", export_data_dict)
        else:
            export_data(export_format, export_data_dict)

### Running the Enhanced Dashboard

To run the dashboard with chatbot integration:

In [ ]:
# Running the Enhanced Dashboard

# 1. Ensure all dependencies are installed
!pip install streamlit boto3 pandas plotly

# 2. Set up AWS credentials (if not already configured)
# aws configure

# 3. Run the dashboard
# In terminal:
# streamlit run finops_dashboard_with_chatbot.py

# 4. Access the dashboard at http://localhost:8501

# Key features to test:
# - Navigate to the AI Chat tab
# - Ask questions about your costs
# - Try the enhanced chat mode toggle in the sidebar
# - Test export functionality
# - Use quick prompts for common queries

# Example queries for the chatbot:
example_queries = [
    "What are my top 5 AWS services by cost?",
    "How much am I spending on EC2 instances?",
    "Show me my cost trend for the last week",
    "Which services are growing fastest?",
    "How can I reduce my AWS costs?",
    "Find underutilized EC2 instances",
    "What's my daily average spend?",
    "Forecast my costs for next month",
    "Are there any cost anomalies?",
    "What percentage of my costs is from S3?"
]

print("Example queries you can ask the chatbot:")
for i, query in enumerate(example_queries, 1):
    print(f"{i}. {query}")

### Integration with MCP Server

The chatbot can also integrate with the MCP (Model Context Protocol) server for enhanced AI capabilities:

In [ ]:
# MCP Integration for Enhanced AI Capabilities

import websocket
import json

class MCPChatbotIntegration:
    """Integrate chatbot with MCP server for enhanced capabilities"""
    
    def __init__(self, mcp_url="ws://localhost:8765"):
        self.mcp_url = mcp_url
        self.ws = None
        self.connect()
    
    def connect(self):
        """Connect to MCP server"""
        try:
            self.ws = websocket.create_connection(self.mcp_url)
            print("Connected to MCP server")
        except Exception as e:
            print(f"Failed to connect to MCP: {e}")
            self.ws = None
    
    def query_mcp_tool(self, tool_name, parameters):
        """Query a specific MCP tool"""
        if not self.ws:
            return {"error": "MCP not connected"}
        
        request = {
            "type": "tool_call",
            "tool": tool_name,
            "parameters": parameters
        }
        
        try:
            self.ws.send(json.dumps(request))
            response = json.loads(self.ws.recv())
            return response
        except Exception as e:
            return {"error": str(e)}
    
    def enhance_chatbot_response(self, user_prompt, base_response):
        """Enhance chatbot response using MCP tools"""
        prompt_lower = user_prompt.lower()
        
        # Determine which MCP tool to use
        if "forecast" in prompt_lower or "predict" in prompt_lower:
            # Use forecasting tool
            mcp_result = self.query_mcp_tool(
                "forecast_costs",
                {"months": 3}
            )
        elif "optimize" in prompt_lower or "save" in prompt_lower:
            # Use optimization tool
            mcp_result = self.query_mcp_tool(
                "get_optimization_recommendations",
                {"resource_type": "all"}
            )
        else:
            # Use general cost analysis
            mcp_result = self.query_mcp_tool(
                "get_cost_analysis",
                {"days": 7}
            )
        
        # Combine MCP insights with base response
        if "error" not in mcp_result:
            enhanced_response = f"{base_response}\\n\\n"
            enhanced_response += "**Additional AI Insights from MCP:**\\n"
            enhanced_response += json.dumps(mcp_result, indent=2)
            return enhanced_response
        
        return base_response

# Usage in the chatbot
def enhanced_chatbot_with_mcp(prompt):
    """Enhanced chatbot that uses both Bedrock and MCP"""
    
    # Get base response from Bedrock or fallback
    base_response = query_bedrock_agent(prompt)
    
    # Try to enhance with MCP if available
    try:
        mcp_integration = MCPChatbotIntegration()
        enhanced_response = mcp_integration.enhance_chatbot_response(
            prompt, 
            base_response
        )
        return enhanced_response
    except:
        # If MCP fails, return base response
        return base_response

# Example usage
if __name__ == "__main__":
    # Test enhanced chatbot
    test_prompt = "What are my optimization opportunities?"
    response = enhanced_chatbot_with_mcp(test_prompt)
    print(response)

### Summary of Chatbot Integration

The integrated chatbot provides:

1. **Natural Language Interface**: Users can ask questions in plain English
2. **Real-time Data Access**: Always uses current AWS cost data
3. **AI-Powered Analysis**: Leverages Bedrock agents for intelligent insights
4. **Fallback Mechanisms**: Works even if AI services are unavailable
5. **Context Awareness**: Maintains conversation history
6. **Export Capabilities**: Can export chat history and insights
7. **MCP Integration**: Can use external AI tools for enhanced capabilities
8. **Enhanced Chat Mode**: Dedicated interface for AI interactions

The chatbot seamlessly integrates with all existing FinOps components:
- Lambda functions for data processing
- Bedrock agents for AI analysis
- MCP server for external tool access
- Real AWS APIs for accurate data

This creates a powerful, unified platform for AI-driven FinOps management.